In [1]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import cpi
import pymongo
from tqdm import tqdm
from scipy import stats
import math
import numpy as np
import pandas as pd
import json

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["imdb"]
imdb = db['imdb_cleaned']

In [3]:
num_movies = imdb.count_documents({})
num_movies

8181

In [4]:
def getMovie(imdb_id):
    for x in imdb.find({'imdb_id': imdb_id}):
        return x
    return None

In [5]:
dir_features = {}
with open('SavedFeatures/director_features.json', 'r') as f:
    dir_features = json.load(f)

In [6]:
len(dir_features.keys())

4078

In [7]:
# nolan = dir_features['nm0634240']
nolan = dir_features['nm0000229']

In [8]:
nolan

{'@type': 'Person',
 'url': '/name/nm0000229/',
 'name': 'Steven Spielberg',
 'year-wise-performance': {'1967': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1968': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1969': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1970': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1971': {'num_movies': 1,
   'sum_rating': 7.6,
   'sum_revenue': 16257,
   'sum_budget': 2875677,
   'num_raters': 64059},
  '1972': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1973': {'num_movies': 0,
   'sum_rating': 0,
   'sum_revenue': 0,
   'sum_budget': 0,
   'num_raters': 0},
  '1974': {'num_movies': 1,
   'sum_rating': 6.8,
   'sum_revenue': 39398284,
   'sum_budget': 15749148,
   'num

In [9]:
rolling_year = 5
present = 2020
first_movie_year = 1967

for year in nolan['year-wise-performance']:
    if(nolan['year-wise-performance'][year]['num_movies'] != 0):
        first_movie_year = int(year)
        break
first_movie_year

1971

In [10]:
def getValues(st_year, nd_year, entity):
    info = {}
    info['total_revenue'] = 0
    info['total_budget'] = 0
    info['num_movies'] = 0
    info['num_raters'] = 0
    info['max_revenue'] = 0
    for year in range(st_year, nd_year+1):
        year = str(year)
        info['total_revenue']   += entity['year-wise-performance'][year]['sum_revenue']
        info['total_budget']    += entity['year-wise-performance'][year]['sum_budget']
        info['num_movies']      += entity['year-wise-performance'][year]['num_movies']
        info['num_raters']      += entity['year-wise-performance'][year]['num_raters']
        info['max_revenue']     = max(info['max_revenue'], entity['year-wise-performance'][year]['sum_revenue'])
    return info

In [11]:
getValues(first_movie_year, present, nolan)

{'total_revenue': 20047233451,
 'total_budget': 2957020796,
 'num_movies': 33,
 'num_raters': 12249844,
 'max_revenue': 2267777508}

In [12]:
year_label = []
revenue_arr = []
budget_arr = []
num_movie_arr = []

for year in range(first_movie_year+rolling_year-1, present):
    st_year = year - rolling_year + 1
    nd_year = year
    info = getValues(st_year, nd_year, nolan)
    year_label.append('{} - {}'.format(st_year, nd_year))

    if(info['num_movies'] != 0):
        revenue_arr.append(info['total_revenue']/info['num_movies'])
        budget_arr.append(info['total_budget']/info['num_movies'])
        num_movie_arr.append(info['num_movies'])
    else:
        revenue_arr.append(0)
        budget_arr.append(0)
        num_movie_arr.append(0) 

In [13]:
def plotFigure(xrr, yrr, zrr):
    fig = make_subplots()
    fig.add_trace(
        go.Scatter(
            x=xrr, 
            y=yrr, 
            name="Revenue",
            mode='lines+markers',
            line_color='rgba(102, 0, 204, .8)'
        ),
    )
    
    fig.add_trace(
        go.Scatter(
            x=xrr, 
            y=zrr, 
            name="Budget",
            mode='lines+markers',
            line_color='rgba(153, 0, 51, .8)'
        ),
    )
    return fig

In [14]:
fig = plotFigure(year_label, revenue_arr, budget_arr)
fig.show()

# Star Power -- KS Test

In [15]:
def getEntityID(entity):
    # print(entity)
    id = entity['url'].split('?')[0].split('/')[-2]
    return id

In [16]:
def getStarInfo(star_id):
    star_info = sta_features[star_id]
    return star_info['total_movies'], star_info['total_raters']

In [17]:
movie = getMovie('tt0371746')
# movie

In [18]:
def getStarInfoBefore(star_id, year):
    return getValues(1967, year-1, difeatures[star_id])

In [19]:
getStarInfo('nm0000229')

NameError: name 'star_features' is not defined

In [36]:
getStarInfoBefore('', 2008)

{'total_revenue': 16974537491,
 'total_budget': 2003547835,
 'num_movies': 25,
 'num_raters': 10314648}

In [42]:
th_num_movie = 5
th_avg_raters = 100000
th_revenue = 100*1000000


def getMovieCriteria(movie, printLog=False):
    # cast_ensamble = movie['cast_and_character']
    if('director' not in movie):
        if(printLog == True):
            print(movie['imdb_id'], "could ont find director")
    else:
        directors = movie['director']
        if(type(directors) is not list):
            directors = [directors]
        for cast in directors:
            star = cast
            star_id = getEntityID(star)
            # num_movie, total_raters = getStarInfo(star_id)
            info = getStarInfoBefore(star_id, int(movie['cleaned_year']))
            num_movie = info['num_movies']
            total_raters = info['num_raters']
            mx_revenue = info['max_revenue']
            avg_raters = 0
            if(num_movie != 0):
                avg_raters = total_raters/num_movie
            # print(star_id, num_movie, th_num_movie, total_raters, total_raters/num_movie, th_avg_raters,  star['name'])
            # print(num_movie > th_num_movie, avg_raters > th_avg_raters)
            # if(num_movie > th_num_movie or avg_raters > th_avg_raters):
            #     return True
            if(num_movie > th_num_movie or mx_revenue > th_revenue):
                # print(mx_revenue, th_revenue)
                return True
    return False

In [43]:
getMovieCriteria(movie)

False

In [44]:
# movie

In [45]:
has_star = []
no_star = []

for movie in imdb.find():
    st = getMovieCriteria(movie)
    revenue = movie['cleaned_Revenue']
    if(st == True):
        has_star.append((movie['imdb_id'], movie['name'], revenue))
    else:
        no_star.append((movie['imdb_id'], movie['name'], revenue))

In [46]:
len(has_star), len(no_star)

(2684, 5497)

In [47]:
has_star_revenue = []
no_star_revenue = []

for movie in has_star:
    has_star_revenue.append(movie[2])

for movie in no_star:
    no_star_revenue.append(movie[2])

In [48]:
len(has_star_revenue), len(no_star_revenue)

(2684, 5497)

In [49]:
from scipy.stats import ks_2samp
import numpy as np

has_star_revenue = np.array(has_star_revenue)
no_star_revenue = np.array(no_star_revenue)

In [50]:
ks_2samp(has_star_revenue, no_star_revenue)

KstestResult(statistic=0.38913225124556494, pvalue=2.3605743035466377e-246)

In [51]:
has_star_revenue.mean(), no_star_revenue.mean()

(194107054.36028317, 55162496.57958887)

In [52]:
has_star_revenue.mean()-no_star_revenue.mean()

138944557.7806943